In [1]:
#importing necessary libraries

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

In [2]:
fake_news = pd.read_csv("Fake.csv")
true_news = pd.read_csv("True.csv")

In [3]:
#fake new data
fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
#true new data
true_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
true_news.shape,fake_news.shape

((21417, 4), (23481, 4))

In [6]:
fake_news['class'] = 0
true_news['class'] = 1

In [7]:
# for manual testing
manual_testing_fake = fake_news.tail(10)
manual_testing_true = true_news.tail(10)

In [8]:
# Removing data
for i in range(23470,23481,1):
    fake_news.drop([i],axis=0,inplace=True)

for i in range(23406,23417,1):
    fake_news.drop([i],axis=0,inplace=True)

In [9]:
manual_testing = pd.concat([manual_testing_true, manual_testing_fake], axis = 0)
manual_testing.to_csv('manual_tesing.csv')

In [10]:
merged_data = pd.concat([true_news,fake_news],axis=0)
merged_data.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [12]:
required_df = merged_data.drop(['title', 'subject', 'date'], axis = 1)
required_df.head()

,text,class
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


In [13]:
required_df = required_df.sample(frac = 1)
required_df.head()

,text,class
6370,WASHINGTON (Reuters) - Donald Trump’s nominee ...,1
13283,Don t mess with daddy s little girl! Ivanka Tr...,0
11401,Is Van Jones on Trump s payroll yet or is he j...,0
20993,KIEV (Reuters) - Ukrainian prosecutors have en...,1
18246,LONDON (Reuters) - Two people were injured on ...,1


In [14]:
required_df.isnull().sum()

text     0
class    0
dtype: int64

# Data Cleaning

In [15]:
def remove_un(data):
    data = data.lower()
    data = re.sub('\[.*?\]','',data)
    data = re.sub('\\W',' ',data)
    data = re.sub('https?://\S+|www.\S+','',data)
    data = re.sub('<.*?>+','',data)
    data = re.sub('[%s]'%re.escape(string.punctuation),'',data)
    data = re.sub('\n','',data)
    data = re.sub('\w*\d\w','',data)
    return data

In [16]:
required_df['text']= required_df['text'].apply(remove_un)

In [17]:
required_df.head()

,text,class
6370,washington reuters donald trump s nominee ...,1
13283,don t mess with daddy s little girl ivanka tr...,0
11401,is van jones on trump s payroll yet or is he j...,0
20993,kiev reuters ukrainian prosecutors have en...,1
18246,london reuters two people were injured on ...,1


# Vectorization

In [18]:
# dependent and independent variables for calculation
x = required_df['text']
y = required_df['class']

In [19]:
x_train, x_test, y_train,y_test= train_test_split(x,y,test_size=0.25, random_state = 42)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
vectorization = TfidfVectorizer()
xv_train  = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

# Logistic Regression

In [22]:
from sklearn.linear_model import LogisticRegression

In [25]:
lr = LogisticRegression()
lr.fit(xv_train,y_train)

LogisticRegression()

In [27]:
lr.score(xv_test, y_test)

0.987521169444692

In [29]:
pred_lr = lr.predict(xv_test)

In [31]:
print(classification_report(y_test,pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5928
           1       0.99      0.99      0.99      5291

    accuracy                           0.99     11219
   macro avg       0.99      0.99      0.99     11219
weighted avg       0.99      0.99      0.99     11219



# Decision Tree Classification

In [32]:
from sklearn.tree import DecisionTreeClassifier

In [33]:
DT  = DecisionTreeClassifier()
DT.fit(xv_train,y_train)

DecisionTreeClassifier()

In [34]:
DT.score(xv_test,y_test)

0.9962563508334076

In [35]:
pred_DT = DT.predict(xv_test)

In [36]:
print(classification_report(y_test,pred_DT))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5928
           1       1.00      1.00      1.00      5291

    accuracy                           1.00     11219
   macro avg       1.00      1.00      1.00     11219
weighted avg       1.00      1.00      1.00     11219



# Random Forest Classifier

In [37]:

from sklearn.ensemble import RandomForestClassifier

In [38]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train,y_train)

RandomForestClassifier(random_state=0)

In [39]:
RFC.score(xv_test,y_test)

0.9898386665478207

In [40]:
pred_RFC = RFC.predict(xv_test)

In [41]:
print(classification_report(y_test,pred_RFC))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5928
           1       0.99      0.99      0.99      5291

    accuracy                           0.99     11219
   macro avg       0.99      0.99      0.99     11219
weighted avg       0.99      0.99      0.99     11219



# Manual Testing

In [42]:
def output_label(n):
    if n ==0:
        return "Fake News"
    elif n==1:
        return "True News"
    else:
        return "ERROR!!"
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test['text'] = new_def_test['text'].apply(remove_un)
    new_x_test = new_def_test['text']
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)
    
    return print(f'''\n
                    LR Prediction: {output_label(pred_LR)}\n
                    DT Prediction: {output_label(pred_DT)}\n
                    GBC Prediction: {output_label(pred_GBC)}\n
                    RFC Prediction: {output_label(pred_RFC)}\n
                 ''')

In [43]:
news = input("News Here")
manual_testing(news)

News HereHello How are you


NameError: name 'LR' is not defined